In [29]:
import pandas as pd

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

In [30]:
merged = pd.read_csv(path+'data/merged.csv')

In [33]:
df = merged.drop(['country', 'lat', 'long', 'date', 'date_of_birth', 'nationality'], axis=1)

In [34]:
### Removing sprint races

df = df[~
(
    ((df.season == 2014) & (df['round'] == 19))
    |
    ((df.season == 2021) & (df['round'].isin([10, 14, 19])))
    |
    ((df.season == 2022) & (df['round'].isin([4, 11, 21])))
)
]

In [38]:
df.rename(columns={'driver_points': 'driver_points_after_race', 
    'constructor_points': 'constructor_points_after_race', 
    'constructor_wins': 'constructor_wins_after_race', 
    'driver_wins': 'driver_wins_after_race', 
    'constructor_standings_pos': 'constructor_standings_pos_after_race',
    'driver_standings_pos': 'driver_standings_pos_after_race'}, inplace=True)

In [47]:
grouped = df.groupby(['season', 'round'])
driver_points_before_race = []
constructor_points_before_race = []

for index, group in grouped:
        for _, row in group.iterrows():
                r = index[1]
                query = df.query(
                        'season ==@index[0] & round ==@r-1 & driver ==@row.driver')
                
                if len(query.driver_points_after_race.values != 0):
                        driver_points_before_race.append(query.driver_points_after_race.values[0])
                        constructor_points_before_race.append(query.constructor_points_after_race.values[0])
                else:
                        driver_points_before_race.append(0.0)
                        constructor_points_before_race.append(0.0)

[0]
[25]
[0]
[12]
[8]
[15]
[0]
[10]
[18]
[2]
[6]
[0]
[0]
[0]
[0]
[0]
[0]
[4]
[0]
[1]
[25]
[43]
[1]
[0]
[18]
[15]
[6]
[14]
[24]
[6]
[3]
[0]
[0]
[0]
[0]
[0]
[23]
[20]
[0]
[0]
[4]
[0]
[50]
[61]
[26]
[12]
[23]
[28]
[18]
[7]
[16]
[3]
[23]
[4]
[20]
[12]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[75]
[79]
[24]
[33]
[24]
[41]
[11]
[0]
[18]
[36]
[23]
[20]
[12]
[4]
[0]
[0]
[0]
[0]
[0]
[0]
[4]
[97]
[100]
[39]
[49]
[37]
[23]
[12]
[4]
[0]
[20]
[17]
[0]
[0]
[0]
[34]
[4]
[0]
[4]
[45]
[20]
[54]
[122]
[45]
[31]
[47]
[61]
[34]
[4]
[21]
[17]
[20]
[18]
[0]
[8]
[4]
[118]
[0]
[0]
[0]
[2]
[140]
[118]
[40]
[18]
[69]
[20]
[23]
[79]
[57]
[18]
[43]
[0]
[0]
[2]
[0]
[0]
[0]
[8]
[60]
[4]
[136]
[55]
[83]
[43]
[60]
[79]
[29]
[59]
[4]
[8]
[28]
[0]
[2]
[0]
[0]
[0]
[165]
[0]
[0]
[30]
[19]
[165]
[73]
[161]
[70]
[87]
[98]
[63]
[55]
[35]
[28]
[19]
[0]
[9]
[0]
[2]
[0]
[0]
[0]
[6]
[8]
[30]
[106]
[97]
[190]
[30]
[19]
[82]
[91]
[9]
[59]
[0]
[6]
[2]
[0]
[0]
[0]
[29]
[69]
[8]
[131]
[202]
[95]
[27]
[88]
[60]
[115]
[29]
[6]
[69]
[11]
[40]
[0]
[

In [48]:
df['driver_points_before_race'] = driver_points_before_race
df['constructor_points_before_race'] = constructor_points_before_race

In [52]:
df[['season', 'round', 'driver', 'driver_points_before_race','driver_points_after_race']].query('season == 2021 & driver == "lewis_hamilton"')

,season,round,driver,driver_points_before_race,driver_points_after_race
2760,2021,1,lewis_hamilton,0.0,25
2781,2021,2,lewis_hamilton,25.0,44
2799,2021,3,lewis_hamilton,44.0,69
2819,2021,4,lewis_hamilton,69.0,94
2845,2021,5,lewis_hamilton,94.0,101
2871,2021,6,lewis_hamilton,101.0,101
2877,2021,7,lewis_hamilton,101.0,119
2895,2021,8,lewis_hamilton,119.0,138
2917,2021,9,lewis_hamilton,138.0,150
2955,2021,11,lewis_hamilton,0.0,195


In [14]:
driver_points_per_race = []
driver_points_from_race = []

constructor_points_per_race = []
constructor_points_from_race = []

for index, row in df.iterrows():
    driver_points_per_race.append(round(row.driver_points / row['round'], 2))
    driver_points_from_race.append(round(row.driver_points_after_race - row.driver_points, 1))

    constructor_points_per_race.append(round(row.constructor_points / row['round'], 2))
    constructor_points_from_race.append(round(row.constructor_points_after_race - row.constructor_points, 1))

In [15]:
df['driver_points_per_race'] = driver_points_per_race
df['driver_points_from_race'] = driver_points_from_race

df['constructor_points_per_race'] = constructor_points_per_race
df['constructor_points_from_race'] = constructor_points_from_race

In [27]:
test = df[['season', 'round', 'driver', 'driver_points', 'driver_points_after_race', 'driver_points_from_race']]

In [28]:
test.query('season == 2021 & driver == "lewis_hamilton"')

,season,round,driver,driver_points,driver_points_after_race,driver_points_from_race
2760,2021,1,lewis_hamilton,0,25.0,25.0
2781,2021,2,lewis_hamilton,25,44.0,19.0
2799,2021,3,lewis_hamilton,44,69.0,25.0
2819,2021,4,lewis_hamilton,69,94.0,25.0
2845,2021,5,lewis_hamilton,94,101.0,7.0
2871,2021,6,lewis_hamilton,101,101.0,0.0
2877,2021,7,lewis_hamilton,101,119.0,18.0
2895,2021,8,lewis_hamilton,119,138.0,19.0
2917,2021,9,lewis_hamilton,138,150.0,12.0
2955,2021,11,lewis_hamilton,177,195.0,18.0


In [17]:
grouped = df.groupby(['season', 'round'])
last_3_races = []

for index, group in grouped:
        for _, row in group.iterrows():
                r= index[1]
                rounds = list(range(r-1, r-4, -1))
                points = df.query(
                        'season ==@index[0] & round in @rounds & driver ==@row.driver'
                        ).agg({
                                'driver_points_from_race': sum
                                }).values[0]

                last_3_races.append(points)

In [18]:
grouped = df.groupby(['season', 'round'])
points_before_race = []

for index, group in grouped:
        for _, row in group.iterrows():
                r = index[1]
                query = df.query(
                        'season ==@index[0] & round ==@r-1 & driver ==@row.driver')

                points = query.driver_points_after_race
                if len(points) != 0:
                        points_before_race.append(points.values[0])
                else:
                        points_before_race.append(0.0)

In [19]:
df['driver_points_last_3_races'] = last_3_races
df['driver_points_before_race'] = points_before_race

In [20]:
testing = df[['season','round', 'driver', 'driver_points',  'driver_points_before_race', 'driver_points_from_race', 'driver_points_last_3_races']]

In [21]:
testing.query('season == 2021 & driver == "max_verstappen"')

,season,round,driver,driver_points,driver_points_before_race,driver_points_from_race,driver_points_last_3_races
2761,2021,1,max_verstappen,0,0.0,18.0,0.0
2780,2021,2,max_verstappen,18,18.0,25.0,18.0
2800,2021,3,max_verstappen,43,43.0,18.0,43.0
2820,2021,4,max_verstappen,61,61.0,19.0,61.0
2839,2021,5,max_verstappen,80,80.0,25.0,62.0
2874,2021,6,max_verstappen,105,105.0,0.0,62.0
2876,2021,7,max_verstappen,105,105.0,26.0,44.0
2894,2021,8,max_verstappen,131,131.0,25.0,51.0
2914,2021,9,max_verstappen,156,156.0,26.0,51.0
2962,2021,11,max_verstappen,185,0.0,2.0,51.0


In [22]:
df[['driver', 'constructor', 
        # 'podium',
    #    'driver_points_after_race', 'driver_wins_after_race',
    #    'driver_standings_pos_after_race', 'constructor_points_after_race',
    #    'constructor_wins_after_race', 'constructor_standings_pos_after_race',
    #    'qualifying_pos',
        # 'starting_grid',
    #    'qual_time', 
       'stage', 'q_delta']].groupby('stage').describe()

q_delta                                                          
        count      mean       std  min      25%     50%     75%     max
stage                                                                  
q1      892.0  3.745922  3.151919  0.0  2.45800  3.0870  3.9390  33.846
q2      896.0  2.295488  1.523904  0.0  1.62125  2.1315  2.5715  26.141
q3     1697.0  1.247106  1.582001  0.0  0.31900  0.9320  1.5930  22.112

In [23]:
df.to_csv(path+'data/processed.csv', index=False)